In [5]:
# importing libraries
import os
import time
import numpy as np
import pyart
import matplotlib.pyplot as plt
import cartopy
import cartopy.crs as ccrs
from netCDF4 import Dataset
import xarray as xr


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



In [6]:
# Path to radar data on my local machine.
radar_path = r'Z:/Matt/work/X_Band/'
rdr_files = os.listdir(radar_path)

In [7]:
# Verifying what files exist in the list of radar files. 
# In this case, I have several tar files that aren't opened.
rdr_files

['Radar_data',
 'gucxprecipradarS2.00.20220314.000031.raw.nc.tar',
 'gucxprecipradarS2.00.20220314.004440.raw.nc.tar',
 'gucxprecipradarS2.00.20220314.012831.raw.nc.tar',
 'gucxprecipradarS2.00.20220314.021215.raw.nc.tar',
 'gucxprecipradarS2.00.20220314.024143.raw.csu.sail-20220314-024143_47667_22_328_RHI.nc',
 'gucxprecipradarS2.00.20220314.024149.raw.csu.sail-20220314-024149_47668_22_330_RHI.nc',
 'gucxprecipradarS2.00.20220314.024155.raw.csu.sail-20220314-024155_47670_22_332_RHI.nc',
 'gucxprecipradarS2.00.20220314.024759.raw.csu.sail-20220314-024759_47699_22_1_PPI.nc',
 'gucxprecipradarS2.00.20220314.024831.raw.csu.sail-20220314-024831_47700_22_2_PPI.nc',
 'gucxprecipradarS2.00.20220314.024903.raw.csu.sail-20220314-024903_47701_22_4_PPI.nc',
 'gucxprecipradarS2.00.20220314.024935.raw.csu.sail-20220314-024935_47702_22_6_PPI.nc',
 'gucxprecipradarS2.00.20220314.025007.raw.csu.sail-20220314-025007_47703_22_8_PPI.nc',
 'gucxprecipradarS2.00.20220314.025039.raw.csu.sail-20220314-025039

Now for actually reading the data file we're looking for. In this case, we want the lowest elevation scan available. The CSU SAIL X-band radar data comes in individual netcdf files with the ending prefix of PPI or RHI. In this case, we want the lowest elevation scan of the PPI. We'll loop through the files in the directory and pull out the file with the name we're looking for. 

In [8]:
for i in sorted(range(len(rdr_files))):
    files = str(rdr_files[i])
    if '_1_PPI.nc' in str(files):
        filename = files
        
        # Joins the data to the path for it to be read
        filename = os.path.join(radar_path, filename)

        # Reads the data in.
        radar = pyart.io.read(filename)

C:\Users\matth\anaconda3\lib\site-packages\pyart\io\cfradial.py:112: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  metadata = dict([(k, getattr(ncobj, k)) for k in ncobj.ncattrs()])
C:\Users\matth\anaconda3\lib\site-packages\pyart\io\cfradial.py:112: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  metadata = dict([(k, getattr(ncobj, k)) for k in ncobj.ncattrs()])
C:\Users\matth\anaconda3\lib\site-packages\pyart\io\cfradial.py:112: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  metadata = dict([(k, getattr(ncobj, k)) for k in ncobj.ncattrs()])
C:\Users\matth\anaconda3\lib\site-packages\pyart\io\cfradial.py:120: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https:

C:\Users\matth\anaconda3\lib\site-packages\pyart\io\cfradial.py:344: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  d = dict((k, getattr(ncvar, k)) for k in ncvar.ncattrs()
C:\Users\matth\anaconda3\lib\site-packages\pyart\io\cfradial.py:344: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  d = dict((k, getattr(ncvar, k)) for k in ncvar.ncattrs()
C:\Users\matth\anaconda3\lib\site-packages\pyart\io\cfradial.py:344: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  d = dict((k, getattr(ncvar, k)) for k in ncvar.ncattrs()
C:\Users\matth\anaconda3\lib\site-packages\pyart\io\cfradial.py:372: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.

In [9]:
        # Checking to make sure it's pulling the correct file name.
        filename

'Z:/Matt/work/X_Band/gucxprecipradarS2.00.20220314.024759.raw.csu.sail-20220314-024759_47699_22_1_PPI.nc'

In [10]:
        # Quickly qc data using velocity texture. This is moreso to filter out noise.
        # The filtering may be aggressive on this, but it's mainly for the plot.
        vel_texture = pyart.retrieve.calculate_velocity_texture(
            radar, vel_field='VEL', wind_size=3, nyq=33.4)
        radar.add_field('velocity_texture', vel_texture, replace_existing=True)
        # Add a gatefilter, and filter using a threshold value of 4 - could be refined
        gatefilter = pyart.filters.GateFilter(radar)
        gatefilter.exclude_above('velocity_texture', 3)

Setting up for Z-S relationship from a github example from Scott Collis. https://github.com/scollis/notebooks/blob/master/urban/Boston%20Snow%20Retrieval.ipynb
Relationship values derived from a paper by Sergey Matrosov. https://journals.ametsoc.org/view/journals/atot/26/11/2009jtecha1238_1.xml#i1520-0426-26-11-2324-e8


In [11]:
        SWE_ratio = 8.5
        snow_z = radar.fields['DBZ']['data'].copy()
        # Convert it from dB to linear units
        z_lin = 10.0**(radar.fields['DBZ']['data']/10.)
        # Apply the Z-S relation, in this case the WSR-88D Northeast relation
        snow_z = SWE_ratio * (z_lin/67)**(1.28)  # a=120 b 2
        # Add the field back to the radar. Use reflectivity as a template
        radar.add_field_like('DBZ', 'snow_z',  snow_z, replace_existing=True)
        # Update units and metadata
        radar.fields['snow_z']['units'] = 'mm/h'
        radar.fields['snow_z']['standard_name'] = 'snowfall_rate'
        radar.fields['snow_z']['long_name'] = 'snowfall_rate_from_z'
        radar.fields['snow_z']['valid_min'] = 0
        radar.fields['snow_z']['valid_max'] = 500
        smax = 60
